<a href="https://colab.research.google.com/github/An0816/Machine_Learning/blob/main/ML_Competition1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv('competition1_train.csv',encoding='cp949')
test = pd.read_csv('competition1_test.csv',encoding='cp949')
y_train = pd.read_csv('competition1_y.csv',encoding='cp949').gender

In [ ]:
tr = pd.concat([train, test])

In [ ]:
# 한 달에 내는 실제 금액
tr['real_amt']= tr.tot_amt / tr.inst_mon

# 날짜 중 필요한 부분만
tr['sdate'] = tr.sales_date.str[:10]

# 할인율
tr['dist_rate'] = (tr['dis_amt']/tr['tot_amt'])*100

In [ ]:
IDtest = test.custid.unique()

In [ ]:
level = 'corner_nm'
# np.where를 통해 ID가 해당 corner에 들린 이력이 있기만 하면 1 아니면 0으로 매핑합니다.
X_train = pd.pivot_table(tr, index='custid', columns=level, values='tot_amt',
                         aggfunc=lambda x: np.where(len(x) >=1, 1, 0), fill_value=0). \
                         reset_index(). \
                         query('custid not in @IDtest') # train set에만 있는 data를 활용합니다.
X_test = pd.pivot_table(tr, index='custid', columns=level, values='tot_amt',
                         aggfunc=lambda x: np.where(len(x) >=1, 1, 0), fill_value=0). \
                         reset_index(). \
                         query('custid in @IDtest') # test set에만 있는 data를 활용합니다.

In [ ]:
features = []

In [ ]:
# 구매 분을 제외하고 시간만 불러옵니다.
tr['구매시간대'] =tr['sales_time']//100 

# 추출한 시간을 활용하여 퇴근 전 후를 구분합니다.
def fx3(x):
    if 9<=x<=12 or 13<=x<=18:
        return('퇴근전')
    else:
        return('퇴근후')
    
tr['구매시간유형'] = tr['구매시간대'].map(fx3)

# 각각 고객이 퇴근 전 후의 시간에 얼마나 많은 구매가 일어나는지를 반영합니다.
a = pd.pivot_table(index='custid',columns='구매시간유형',values='tot_amt',aggfunc=np.size,
                   fill_value=0,data = tr.query('tot_amt>0')[['custid','구매시간유형','tot_amt']]).reset_index()

In [ ]:
#고객의 전체 구매 건수를 나타냅니다.
all_amt = tr.query("tot_amt>0").groupby('custid')['tot_amt'].size()

In [ ]:
# 고객의 전체 구매 건수를 나눠 퇴근 전 후의 구매 비율을 표현합니다.
f = a.drop('custid',axis=1).apply(lambda x: x/ all_amt).rename_axis('custid').reset_index()
features.append(f)

In [ ]:
# 고객이 퇴근 전, 퇴근 후에 얼마나 많은 금액을 사용하는 지를 변수로 반영합니다.
tr['구매시간유형1'] = tr['구매시간유형']+str(1)

a = tr.query("net_amt>0").groupby(['custid','구매시간유형1'])['net_amt'].agg([('시간별구매액합',sum)]).reset_index()
a = pd.pivot_table(index='custid',columns='구매시간유형1',values='시간별구매액합',fill_value=0,aggfunc=sum,data=a).reset_index()
features.append(a)
a.head(1)

구매시간유형1,custid,퇴근전1,퇴근후1
0,0,874800,693000


In [ ]:
# 고객의 전체 구매액
b = tr.query("net_amt>0").groupby('custid')['net_amt'].agg([('구매액합',sum)]).reset_index()['구매액합']
# 고객이 퇴근 전, 퇴근 후에 얼마나 많은 금액을 사용하는 지를 비율로 표현해 변수로 반영합니다.
a = a.iloc[:,1:3].apply(lambda x: x/b).rename_axis('custid').reset_index()
a.columns = ['custid','퇴근전비율','퇴근후비율']
features.append(a)
a.head(2)

,custid,퇴근전비율,퇴근후비율
0,0,0.557979,0.442021
1,1,0.996454,0.003546


In [ ]:
a = tr[tr.corner_nm.str.contains('남성')]
b = a.groupby('custid')['part_nm'].agg([('남성코너구매횟수', np.size)]).reset_index()
c = a.groupby('custid')['tot_amt'].agg([('남성코너구매합', np.sum)]).reset_index()
c['남성코너구매여부'] = 1  # c에 있는 custid들에 대해서 남성 코너 구매여부를 표현합니다.
features.append(b)
features.append(c)

In [ ]:
f = tr.groupby('custid')['tot_amt'].agg([('총구매액', 'sum')]).reset_index()
features.append(f)

In [ ]:
f = tr.groupby('custid')['tot_amt'].agg([('평균구매가격', 'mean')]).reset_index()
features.append(f)

In [ ]:
f = tr.groupby('custid')['inst_mon'].agg([('평균할부개월수', 'mean')]).reset_index()
f.iloc[:,1] = f.iloc[:,1].apply(round, args=(1,))
features.append(f)

In [ ]:
tr['sdate'] = tr.sales_date.str[:10]
f = tr.groupby(by = 'custid')['sdate'].agg([('내점일수','nunique')]).reset_index()
features.append(f)

In [ ]:
for feat in features : 
    X_train = pd.merge(X_train,feat,how='left',on=['custid'])
    X_test = pd.merge(X_test,feat,how='left',on=['custid'])

In [ ]:
X_train = X_train.drop(columns=['custid'])
X_test = X_test.drop(columns=['custid'])

In [ ]:
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_auc_score
auc = make_scorer(roc_auc_score, greater_is_better = True, needs_proba = True)

In [ ]:
import scipy as sp

In [ ]:
sp.random.seed(0)

In [ ]:
parameters = {
    "learning_rate": sp.stats.uniform(),
    "max_depth":sp.stats.randint(5,80),
    "n_estimators":sp.stats.randint(30,300)
    }

In [ ]:
from lightgbm import LGBMClassifier
model = LGBMClassifier(random_state = 1218, n_jobs = -1)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
rand = RandomizedSearchCV(model, parameters, n_jobs = -1, cv = 4, scoring = auc, verbose = 2)

In [ ]:
%%time
rand.fit(np.array(X_train), np.array(y_train))

Fitting 4 folds for each of 10 candidates, totalling 40 fits
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Wall time: 21.8 s


RandomizedSearchCV(cv=4, estimator=LGBMClassifier(random_state=1218), n_jobs=-1,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000017DBCFECA00>,
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000017DBCFECC10>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000017DBCFECD30>},
                   scoring=make_scorer(roc_auc_score, needs_proba=True),
                   verbose=2)

In [ ]:
rand.best_estimator_

LGBMClassifier(learning_rate=0.08712929970154071, max_depth=74,
               n_estimators=129, random_state=1218)

In [ ]:
rand.best_score_

0.6620278397848605

In [ ]:
from lightgbm import LGBMClassifier
model = LGBMClassifier(random_state = 1218, learning_rate = 0.08712929970154071, max_depth=74, n_estimators = 129, objective = 'binary', subsample = 0.9, colsample_bytree = 0.9, n_jobs = -1, booster = 'dart')

In [ ]:
model.fit(np.array(X_train), np.array(y_train))

LGBMClassifier(booster='dart', colsample_bytree=0.9,
               learning_rate=0.08712929970154071, max_depth=74,
               n_estimators=129, objective='binary', random_state=1218,
               subsample=0.9)

In [ ]:
pred = model.predict_proba(X_test)[:,1]

이렇게 해서 base_lgbm5.csv 제출

### Final Submission 
- GridSearchCV를 통해서 best parameters를 찾는 것에 집중  
- 1st : LightGBM부터 손질 (GridSearch만 사용)
- 2nd : XGBoost 손질(RandomizedSearchCV + GridSearchCV 사용)

In [ ]:
from lightgbm import LGBMClassifier

model = LGBMClassifier(random_state = 1218, n_jobs = -1)

In [ ]:
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_auc_score

auc = make_scorer(roc_auc_score, greater_is_better = True, needs_proba = True)

In [ ]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(model,
                   {
                    'learning_rate' : [0.01, 0.05, 0.1, 0.2],
                    'max_depth' : [5, 7, 9, 10, 30],
                    'n_estimators': [100, 300, 600]
                   },
                    n_jobs = -1,
                    cv = 4,
                    scoring = auc,
                    verbose = 2
                   )

In [ ]:
%%time
grid.fit(np.array(X_train), np.array(y_train))

Fitting 4 folds for each of 60 candidates, totalling 240 fits
Wall time: 2min 1s


GridSearchCV(cv=4, estimator=LGBMClassifier(random_state=1218), n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.05, 0.1, 0.2],
                         'max_depth': [5, 7, 9, 10, 30],
                         'n_estimators': [100, 300, 600]},
             scoring=make_scorer(roc_auc_score, needs_proba=True), verbose=2)

In [ ]:
grid.best_params_

{'learning_rate': 0.01, 'max_depth': 30, 'n_estimators': 600}

In [ ]:
from sklearn.model_selection import GridSearchCV

grid2 = GridSearchCV(model,
                    {
                    'subsample' : [0.5, 0.7, 0.9],
                    'subsample_freq' : [1, 3, 5],
                    'colsample_bytree' : [0.5, 0.7, 0.9]
                    },
                    n_jobs = -1,
                    cv = 4,
                    scoring = auc,
                    verbose = 2
                    )

In [ ]:
%%time
grid2.fit(np.array(X_train), np.array(y_train))

Fitting 4 folds for each of 27 candidates, totalling 108 fits
Wall time: 28.7 s


GridSearchCV(cv=4, estimator=LGBMClassifier(random_state=1218), n_jobs=-1,
             param_grid={'colsample_bytree': [0.5, 0.7, 0.9],
                         'subsample': [0.5, 0.7, 0.9],
                         'subsample_freq': [1, 3, 5]},
             scoring=make_scorer(roc_auc_score, needs_proba=True), verbose=2)

In [ ]:
grid2.best_params_

{'colsample_bytree': 0.5, 'subsample': 0.9, 'subsample_freq': 1}

In [ ]:
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(random_state = 1218, n_jobs = -1, learning_rate = 0.01, max_depth = 30, n_estimators = 600, subsample = 0.9, subsample_freq = 1, colsample_bytree = 0.5, boosting_type = 'dart')

In [ ]:
lgbm.fit(np.array(X_train), np.array(y_train))

LGBMClassifier(boosting_type='dart', colsample_bytree=0.5, learning_rate=0.01,
               max_depth=30, n_estimators=600, random_state=1218, subsample=0.9,
               subsample_freq=1)

In [ ]:
lgbm_pred = lgbm.predict_proba(np.array(X_test))[:,1]

#### XGBoost

In [ ]:
from xgboost import XGBClassifier
model = XGBClassifier(random_state = 1218, n_jobs = -1)

In [ ]:
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_auc_score
auc = make_scorer(roc_auc_score, greater_is_better = True, needs_proba = True)

In [ ]:
parameters = {
    'learning_rate' : [0.01, 0.05, 0.1, 0.2],
    'max_depth' : [5, 7, 9, 12],
    'n_estimators' : [30, 100, 300, 600]
}

In [ ]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(model, parameters, n_jobs = -1, cv = 4, scoring = auc, verbose = 2)

In [ ]:
%%time
grid.fit(np.array(X_train), np.array(y_train))

Fitting 4 folds for each of 64 candidates, totalling 256 fits


C:\Users\lg\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:13:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Wall time: 1h 21min 32s


GridSearchCV(cv=4,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=-1,
                                     num_parallel_tree=None, random_state=1218,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, subsample=None,
                                     tree_method=None, v

In [ ]:
grid.best_params_

{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}

#### RandomizedSearchCV
- Random Sampling을 통해서 parameter를 설정

In [ ]:
import scipy as sp # 수학 연산에 사용되는 파이썬 라이브러리 

In [ ]:
sp.random.seed(0)

In [ ]:
parameters = {
    'learning_rate' : sp.stats.uniform(), 
    'max_depth' : sp.stats.randint(5, 100), # 
    'n_estimators' : sp.stats.randint(30, 600)
}

In [ ]:
parameters = {
    'learning_rate' : sp.stats.uniform(), # [0, 1] 사이의 균일한 연속변수를 생성(random하게 추출)
    'max_depth' : [int(x) for x in range(5, 100)], 
    'n_estimators' : [int(x) for x in range(30, 600, 10)]
}

In [ ]:
from xgboost import XGBClassifier
model = XGBClassifier(random_state = 1218, n_jobs = -1)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
rand = RandomizedSearchCV(model, parameters, n_jobs = -1, cv = 5, scoring = auc, verbose = 2)

In [ ]:
%%time
rand.fit(np.array(X_train), np.array(y_train))

Fitting 5 folds for each of 10 candidates, totalling 50 fits


C:\Users\lg\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[01:11:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Wall time: 1h 14min 43s


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100,...
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000017DB3829070>,
                                        'max_dep

In [ ]:
rand.best_score_

0.6425167264509888

In [ ]:
rand.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.06022547162926983, max_delta_step=0, max_depth=40,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=410, n_jobs=-1, num_parallel_tree=1,
              random_state=1218, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

RandomizedSearchCV 후 GridSearchCV

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier(random_state = 1218, max_depth = 14, learning_rate = 0.07103605819788694, objective = 'binary:logistic', subsample = 0.9, colsample_bytree = 0.9, n_estimators = 145, n_jobs = -1, booster = 'dart')

In [ ]:
xgb.fit(np.array(X_train), np.array(y_train))

In [ ]:
xgb_pred = xgb.predict_proba(X_test)[:,1]

In [ ]:
pred = (lgbm_pred + xgb_pred) / 2

#### 그 밖에 시도한 것들 (in case of 변수 생성)
1. 주구매코너 -> label Encoding
2. 요일별 통계 (합계, 비율) 했더니 성능 하락..?
3. 지점별 통계 (합계, 비율)

In [ ]:
from collections import Counter

df = data.groupby(['custid'])['corner_nm'].transform(lambda x: Counter(x).most_common(1)[0][0])
# most_common은 (가장 많이 나온거, 개수) 이렇게 tuple형태로 리턴

In [ ]:
tmp1 = data.groupby(['custid'])['corner_nm'].agg(lambda x: Counter(x).most_common(1)[0][0]).reset_index()

In [ ]:
tmp1.rename(columns = {'corner_nm': '주구매코너'}, inplace = True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
tmp1['주구매코너'] = le.fit_transform(tmp1['주구매코너'])

#### 그 밖에 시도한 것들
1. RandomForest를 사용 -> 성능이 별로 (0.63..., 0.50...)
2. TPOT도 썼는데 0.499..?


In [ ]:
from lightgbm import LGBMClassifier

model = LGBMClassifier(random_state = 1218, n_jobs = -1)

In [ ]:
parameters = {
    'subsample' : [0.5, 0.7, 0.9],
    'colsample_bytree' : [0.5, 0.7, 0.9]
}

In [ ]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(model, parameters, n_jobs = -1, cv = 4, verbose = 2, scoring = auc)

In [ ]:
%%time
grid.fit(np.array(X_train), np.array(y_train))

Fitting 4 folds for each of 9 candidates, totalling 36 fits
Wall time: 9.31 s


GridSearchCV(cv=4, estimator=LGBMClassifier(random_state=1218), n_jobs=-1,
             param_grid={'colsample_bytree': [0.5, 0.7, 0.9],
                         'subsample': [0.5, 0.7, 0.9]},
             scoring=make_scorer(roc_auc_score, needs_proba=True), verbose=2)

In [ ]:
grid.best_params_

{'colsample_bytree': 0.5, 'subsample': 0.5}

In [ ]:
grid.best_score_

0.6659455875292443

In [ ]:
from lightgbm import LGBMClassifier

model1 = LGBMClassifier(random_state = 1218, learning_rate = 0.01, max_depth = 12, objective = 'binary', subsample = 0.5, colsample_bytree = 0.5, n_estimators = 600, n_jobs = -1, booster = 'dart')